In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

# Replace this URL with the one you want to scrape
# url = "https://www.ifsc-climbing.org/index.php/world-competition"
# url = "https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1291"
url = "https://components.ifsc-climbing.org/calendar/"
driver = webdriver.Chrome(executable_path='chromedriver')
bern_url = "https://bern2023.org/en/programme/"
driver.get(url)

time.sleep(5)
soup = BeautifulSoup(driver.page_source, 'html.parser')


In [12]:
event_urls = []
for tag in soup.find_all('div', class_="competition"):
    for a in tag.find_all('a'):
        if a.text == 'Event Page':
            event_urls.append(a['href'])

In [13]:
event_urls

['https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1291',
 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1292',
 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1293',
 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1294',
 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1295',
 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1296',
 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1297',
 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1298',
 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1299',
 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1300',
 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1301',
 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1302',
 'https://www.if

In [99]:
import pickle
from urllib.parse import urlparse
from urllib.parse import parse_qs
for event_url in event_urls:
    driver.get(event_url)
    time.sleep(5)
    event_page = BeautifulSoup(driver.page_source, 'html.parser')
    title = event_page.find_all('h1', class_='event_name')[0].text

    event_dict = {}
    flag = True
    event_dict['URL'] = event_url

    if event_page.find('h3', class_='geo'):
        place_url = urlparse(event_page.find('h3', class_='geo').next.next.next['href'])
        event_dict['PLACE'] = parse_qs(place_url.query)
    
    for p in event_page.find_all('p'):
        if p.text.find('SCHEDULE') != -1: flag = False
        if p.text.find('RESULT') != -1: flag = True
        if flag: continue

        spans = p.find_all('span', limit=1)
        if len(spans) < 1: continue
        content = spans[0].text.strip(':')
        if content.find('SCHEDULE') !=-1:
            event_dict['SCHEDULE'] = content[content.find('SCHEDULE')+9:]
        else:
            event_dict[content] = []

        if p.em:
            for em in p.find_all('em'):
                event_dict[content].append([em.text, title + '- ' + em.next.next.strip(' •')[3:]])

    print(event_dict)
    with open('files/'+title+'.pickle', 'wb') as f:   
        pickle.dump(event_dict, f, pickle.HIGHEST_PROTOCOL)

{'URL': 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1291', 'PLACE': {'mlat': ['35.64007319193447'], 'mlon': ['139.29218501646352']}, 'SCHEDULE': '(UTC+9:00)', 'FRIDAY, 21 APRIL': [['9:00', ' IFSC - Climbing World Cup (B) - Hachioji (JPN) 2023 - BOULDER QUALIFICATIONS']], 'SATURDAY, 22 APRIL': [['11:00', " IFSC - Climbing World Cup (B) - Hachioji (JPN) 2023 - WOMEN'S BOULDER SEMI-FINAL"], ['17:00', " IFSC - Climbing World Cup (B) - Hachioji (JPN) 2023 - WOMEN'S BOULDER FINAL"]], 'SUNDAY, 23 APRIL': [['11:00', " IFSC - Climbing World Cup (B) - Hachioji (JPN) 2023 - MEN'S BOULDER SEMI-FINAL"], ['17:00', " IFSC - Climbing World Cup (B) - Hachioji (JPN) 2023 - MEN'S BOULDER FINAL"]]}
{'URL': 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1292', 'PLACE': {'mlat': ['37.5760644498347'], 'mlon': ['127.09707039795447']}, 'SCHEDULE': '(UTC+9:00)', 'FRIDAY, 28 APRIL': [['12:15', ' IFSC - Climbing World Cup (B,S) - Seoul (KOR) 2023 - SPEED QU

In [102]:
import os
events = []
path = './files/'
for f in os.listdir(path):
    with open(path+f, 'rb') as ff:
        events.append(pickle.load(ff))

[{'URL': 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1298',
  'PLACE': {'mlat': ['46.30181040017884'], 'mlon': ['7.051609515247656']},
  'SCHEDULE': '(UTC+2:00)',
  'FRIDAY, 30 JUNE': [['13:00',
    ' IFSC - Climbing World Cup (L,S) - Villars (SUI) 2023 - LEAD QUALIFICATIONS']],
  'SATURDAY, 1 JULY': [['12:00',
    ' IFSC - Climbing World Cup (L,S) - Villars (SUI) 2023 - LEAD SEMI-FINALS'],
   ['20:00',
    ' IFSC - Climbing World Cup (L,S) - Villars (SUI) 2023 - LEAD FINALS']],
  'SUNDAY, 2 JULY': [['12:00',
    ' IFSC - Climbing World Cup (L,S) - Villars (SUI) 2023 - SPEED QUALIFICATIONS'],
   ['18:00',
    ' IFSC - Climbing World Cup (L,S) - Villars (SUI) 2023 - SPEED FINALS']]},
 {'URL': 'https://www.ifsc-climbing.org/index.php/component/ifsc/?view=event&WetId=1291',
  'PLACE': {'mlat': ['35.64007319193447'], 'mlon': ['139.29218501646352']},
  'SCHEDULE': '(UTC+9:00)',
  'FRIDAY, 21 APRIL': [['9:00',
    ' IFSC - Climbing World Cup (B) - Hachioji (JPN)

In [144]:
month_number = {
    'APRIL': 4,
    'MAY': 5,
    'JUNE': 6,
    'JULY': 7,
    'Aug': 8,
    'SEPTEMBER': 9
}


In [175]:
from arrow import Arrow
import datetime
from ics import Calendar, Event, Geo, DisplayAlarm
year = 2023
c = Calendar()
for event in events:
    if 'SCHEDULE' in event:
        tzinfo = event['SCHEDULE'][1:-1]
        url = event['URL']
        geo = Geo(float(event['PLACE']['mlat'][0]), float(event['PLACE']['mlon'][0]))

        for k in event.keys():
            if k == 'URL' or k == 'SCHEDULE' or k == 'PLACE': continue
            try:
                _, day, mon = k.split(',')[1].split(' ')
                day = int(day)
                mon = month_number[mon]
            except Exception:
                continue

            for tt, name in event[k]:
                if tt == 'TBD': 
                    """ make all day"""
                    begin = Arrow(year, mon, day, tzinfo = tzinfo)
                    e = Event(name, begin, url=url, geo=geo)
                    e.make_all_day()
                    continue

                hh, mm = map(int, tt.split(':'))
                begin = Arrow(year, mon, day, hh, mm, tzinfo=tzinfo)
                duration = datetime.timedelta(hours=2)
                if name.find('QUALIFICATION') != -1 and name.find('SPEED') == -1:
                    duration = datetime.timedelta(hours=7)

                e = Event(name, begin, duration=duration, url=url, geo=geo)
                e.location = name.split(' - ')[2][:-5]
                e.alarms = [DisplayAlarm(trigger=datetime.timedelta(minutes=-15))]
                c.events.add(e)


In [134]:
import time
driver = webdriver.Chrome(executable_path='chromedriver')

driver.get(bern_url)
time.sleep(5)
bern_event_page = BeautifulSoup(driver.page_source, 'html.parser')



In [176]:
title = 'IFSC - CLIMBING WORLD CHAMPIONSHIPS (B,L,S,B&L) - BERN (SUI) 2023'
for i in bern_event_page.find_all('h3'):
    if i.contents[0].find('Aug')!=-1:
        mon, day = i.contents[0].split(' ')
        mon = month_number[mon]
        day = int(day)
        print(mon,day)
        p = i.find_parent()
        timeline = p.find('div', class_='fruwo-timeline-time').text
        name =  p.find('div', class_='fruwo-timeline-content').text.replace('\n', ' ')
        try:
            begin, end = timeline.split('-')
            hh, mm = map(int, begin.split(':'))
            begin = Arrow(year, mon, day, hh, mm, tzinfo='UTC+2:00')
            hh, mm = map(int, end.split(':'))
            end = Arrow(year, mon, day, hh, mm, tzinfo='UTC+2:00')
            e = Event(title+ ' - ' + content, begin, end, url='https://bern2023.org/en/')
            e.location = 'PostFinance Arena & Curling Hall Bern'
            e.alarms = [DisplayAlarm(trigger=datetime.timedelta(minutes=-15))]
            c.events.add(e)

        except Exception:
            continue

8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12


In [177]:
with open('ifsc-calender.ics', 'w') as f:
    f.writelines(c.serialize_iter())